In [ ]:
# Load Packages
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle

In [ ]:
# Load dataset
initial_data = pd.read_csv('./data/algebra_2005_2006_train.txt', delimiter = "\t")
print(initial_data.shape)
initial_data.head()

In [ ]:
# rename some columns
initial_data = initial_data.rename(columns={'Anon Student Id': 'user_id','Problem Name': 'item_id','Step Name': 'step_id','KC(Default)': 'kc_id','First Transaction Time': 'timestamp','Correct First Attempt': 'correct',"Hints":'nb_hint','Incorrects':'incorrect','Corrects':'incorrect_?'})
needed = initial_data[["user_id","item_id","timestamp","correct","nb_hint"]]

In [ ]:
needed['timestamp'] = pd.to_datetime(needed['timestamp']).map(lambda t: t.timestamp()).round().astype(np.int32)

# Conservez uniquement les utilisateurs avec au moins 10 interactions
needed = needed.groupby("user_id").filter(lambda x: len(x) >= 10)

# Supprimer les entrées où le résultat n'est pas binaire mais continu
needed = needed[needed.correct.isin([0, 1])]
needed['correct'] = needed['correct'].astype(np.int32)

# Trier la colonne "timestamp" et changer le nom de la colonne
needed = needed.sort_values('timestamp')
print(needed.shape)
needed.head()

In [ ]:
# Encodage de la colonne user_id et item_id
userId_encoder = LabelEncoder()
itemId_encoder = LabelEncoder()

needed["user_id"] = userId_encoder.fit_transform(needed["user_id"])
needed["item_id"] = itemId_encoder.fit_transform(needed["item_id"])

# Save user_id and item_id encoder
####
outputUserInEncoder = open('userId_encoder.pkl', 'wb')
pickle.dump(userId_encoder, outputUserInEncoder)
outputUserInEncoder.close()
####
outputItemInEncoder = open('itemId_encoder.pkl', 'wb')
pickle.dump(itemId_encoder, outputItemInEncoder)
outputItemInEncoder.close()

In [ ]:
# replace user_id and item_id where id == 0 , Pour eviter d'avoir un probleme avec le modèle IRT
needed["user_id"].replace(0,len(needed['user_id'].unique()),inplace=True)
needed["item_id"].replace(0,len(needed['item_id'].unique()),inplace=True)

In [ ]:
# Création de la colonne « answers_using_hint » pour éviter d’avoir des problèmes avec le modèle IRT.
conditions = [
    (needed['correct'] == 1) & (needed['nb_hint'] > 0),  # 1 ==> Correct avec hint
    (needed['correct'] == 1) & (needed['nb_hint'] == 0), # 2 ==> Correct sans hint
    (needed['correct'] == 0) & (needed['nb_hint'] > 0),  # 3 ==> Incorrect avec hint
    (needed['correct'] == 0) & (needed['nb_hint'] == 0)  # 4 ==> Incorrect sans hint
    ]
    
values = [1, 2, 3, 4]

# create a new column and use np.select to assign values to it using our lists as arguments
needed['answers_using_hint'] = np.select(conditions, values)
needed.head()